In [1]:
#Project: Import csv to bigquery table
#StartDate: 4/06/2022
#EndDate: 
#Developer: Bradley, Hongquy, Khoa

In [2]:
from flask import Flask,render_template,request,redirect,url_for
from google.cloud import bigquery
from google.oauth2 import service_account
from google.cloud import storage
import os, re, datetime, uuid, time, json
import google.cloud.logging
import apache_beam as beam
from io import StringIO
import argparse
from apache_beam.options.pipeline_options import PipelineOptions
from werkzeug.utils import secure_filename

In [7]:
#this is to allow a authenticated user on the project to work on bigquery
Key_path = r"C:\Users\Khoa\Documents\Python Scripts\flask-upload-345423-4f033f10d5f3.json"

#this is needed to request a job
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = Key_path
credentials = service_account.Credentials.from_service_account_file(Key_path,scopes=["https://www.googleapis.com/auth/cloud-platform"])
client=bigquery.Client(credentials=credentials,project=credentials.project_id)
#table_id='model-craft-342921.testing.Task3'
storage_client=storage.Client(credentials=credentials,project=credentials.project_id)
bucket = storage_client.get_bucket("sample_flask_ktd")
client1 = google.cloud.logging.Client()
logger = client1.logger(name="dataflow.googleapis.com%2Fworker")



In [8]:
class formating(beam.DoFn):
    def process(self, element, table_sch):
        import apache_beam as beam
        record = {}
        for x in range(0, len(element)):
            name = table_sch[x]['name']
            record[name] = element[x]
        yield record

In [9]:
class process_message(beam.DoFn):
    def process(self, element, bad_input_data, count):
        import apache_beam as beam
        from io import StringIO
        import json, re
        bad_message = element.replace("'",'"')
        bad_message = json.loads(bad_message)
        for input_data in bad_input_data['elements']:
            message = ''
            if input_data[bad_message['location']] == bad_message['message'].split(': ')[1]:
                message = 'Error #{} - Message: {} in column {} - Input: {}\n'.format(count, bad_message['message'],bad_message['location'],input_data)
                count = count + 1
                bad_input_data['elements'].remove(input_data)
                yield message

In [10]:
#Assume that the pipeline is already ran, with the timing already there

In [15]:
beam_options = PipelineOptions(
                                runner = 'DataflowRunner',
                                #runner='DirectRunner',
                                project = 'flask-upload-345423',
                                job_name = 'example',
                                temp_location = 'gs://sample_flask_ktd/temp',
                                region='us-central1',
                                maxNumWorkers= 1,
                                service_account_email = 'example@flask-upload-345423.iam.gserviceaccount.com'
                            )

SCHEMA = {}
table_sch = []
try:
    SchemaJob = client.get_table('flask-upload-345423.sample.{}'.format('email'))
    for s in SchemaJob.schema:
        new_dict = {}
        new_dict['name'] = s.name
        new_dict['type'] = s.field_type
        new_dict['mode'] = s.mode
        table_sch.append(new_dict)
    SCHEMA['fields'] = table_sch

except Exception as e:
    print(e)

start = time.time() # ADD THIS
p = beam.Pipeline(options=beam_options)
events  = (p | 'ReadData' >> beam.io.ReadFromText('gs://sample_flask_ktd/email.csv', skip_header_lines =1)
             | 'Split' >> beam.Map(lambda x: x.split(','))
             | 'format to dict2' >> beam.ParDo(formating(),table_sch) 
             | 'WriteToBigQuery2' >>  beam.io.gcp.bigquery.WriteToBigQuery(
               'flask-upload-345423:sample.{}'.format('email'),
               schema=SCHEMA,
               write_disposition=beam.io.BigQueryDisposition.WRITE_APPEND,
               insert_retry_strategy=beam.io.gcp.bigquery_tools.RetryStrategy.RETRY_NEVER,
               method='STREAMING_INSERTS')
            )
(events[beam.io.gcp.bigquery.BigQueryWriteFn.FAILED_ROWS]
             | "Bad lines" >> beam.io.WriteToText('gs://sample_flask_ktd/{}'.format('email'), shard_name_template = "")
)

result = p.run()
result.wait_until_finish()
end = time.time() # ADD THIS

In [36]:
# Get the time (in seconds) the DataFlow pipeline takes. 
time_elasped = end-start
#print(time_elasped)

# Good elements - read number of elements in BigQuery table
query = "SELECT count(*) FROM `flask-upload-345423.sample.email`"
try:
    results =  client.query(query)
except Exception as e:
        print("ERROR: could not query")
num_good = ""
for row in results:
    num_good = row["f0_"]
#print(num_good)

# Bad elements - get number of rows from the error text file
blob = bucket.get_blob('error.txt')  #Make sure that this points to the error file
downloaded_file = blob.download_as_text(encoding="utf-8")
num_bad = downloaded_file.count('\n')


#Write to GCP bucket
msg = '''Time elasped: {} seconds
Number of Valid Rows: {}
Number of Invalid Rows: {}
'''.format(time_elasped, num_good, num_bad)

blob = bucket.blob("metrics") #Prob change the output location
blob.upload_from_string(msg)

#print(msg)
#OUTPUT:
#Time elasped: 378.0396316051483 seconds
#Number of Valid Rows: 11
#Number of Invalid Rows: 41